# IPython & Jupyter: a PyCon 2017 tutorial

In [ ]:
Class outline:

* A quick installation check of [ipython](https://ipython.org/install.html) and [jupyter notebook](https://jupyter.readthedocs.io/en/latest/install.html)
* An overview of the IPython project from [the official website](http://ipython.org), and [jupyter](https://jupyter.org)
* Super basic intro to the notebook: typing code.
* [Notebook Basics](examples/Notebook/Notebook%20Basics.ipynb)
* [IPython - beyond plain python](examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb)
* [Markdown Cells](examples/Notebook/Working%20With%20Markdown%20Cells.ipynb)
* [Rich Display System](examples/IPython%20Kernel/Rich%20Output.ipynb)
* [Custom Display logic](examples/IPython%20Kernel/Custom%20Display%20Logic.ipynb)
* [Customizing IPython - a condensed version](exercises/Customization/Condensed.ipynb)
* [Running a Secure Public Notebook Server](examples/Notebook/Running%20the%20Notebook%20Server.ipynb#Securing-the-notebook-server)
* [How Jupyter/IPython works](examples/Notebook/Multiple%20Languages%2C%20Frontends.ipynb) to run code in different languages.

Get this tutorial:

    git clone https://github.com/ipython/ipython-in-depth

Install IPython and Jupyter:

with [conda](https://www.continuum.io/downloads):

    conda install ipython jupyter

with pip:

    # first, upgrade pip!
    pip install --upgrade pip
    pip install --upgrade ipython jupyter

Start the notebook in the tutorial directory:

    cd ipython-in-depth
    jupyter notebook

There are a lot more detailed notebooks in this same directory that cover other topics, but we can not cover all in a 3-hour tutorial. We encourage you to explore them and practice on your own.